In [1]:
import imp
import json
import os
import pickle
import milsed
import numpy as np
import math

Using TensorFlow backend.


In [2]:
cd ~/dev/milsed/models/

/home/js7561/dev/milsed/models


In [3]:
# SETUP STUFF
OUTPUT_PATH = '/home/js7561/dev/milsed/models/resources/'
train = imp.load_source('train', '/home/js7561/dev/milsed/models/02_train.py')
pump = pickle.load(open('/home/js7561/dev/milsed/models/resources/pump.pkl', 'rb'))
with open(os.path.join(OUTPUT_PATH, 'index_test.json'), 'r') as fp:
    test_idx = json.load(fp)['id']
pumpfolder = '/beegfs/js7561/datasets/dcase2017/task4_official/combined/features_silence/'
labelfile = ('/beegfs/js7561/datasets/dcase2017/task4_official/combined/metadata/labels/'
             '/groundtruth_strong_label_testing_set.csv')
duration = 10.0

In [4]:
def create_ensemble(model_list, ensemble_version, use_orig_duration=True, save=True):
    # Compute results
    models = []
    for version in model_list:
        weight_path = os.path.join(OUTPUT_PATH, version, 'model.h5')
        params = json.load(open(os.path.join(OUTPUT_PATH, version, 'params.json'), 'r'))
        model, inputs, outputs = train.MODELS[params['modelname']](pump, alpha=1.0)
        model.load_weights(weight_path)
        models.append(model)
        
    print('Computing results:')
    results, results_wfs = milsed.eval.score_ensemble_wfs(
        OUTPUT_PATH, pump, models, test_idx, pumpfolder, labelfile, duration, ensemble_version, 
        use_tqdm=True, use_orig_duration=use_orig_duration, save_jams=save)
        
    # Save them to disk
    if save:
        resultsfile = os.path.join(OUTPUT_PATH, ensemble_version, 'results.json')
        with open(resultsfile, 'w') as fp:
            json.dump(results, fp, indent=2)
            
        resultsfile_wfs = os.path.join(OUTPUT_PATH, ensemble_version, 'predictions_weakfromstrong', 'results.json')
        with open(resultsfile_wfs, 'w') as fp:
            json.dump(results_wfs, fp, indent=2)
        
    return results, results_wfs

## Create final set of ensembles using all good models

In [5]:
from itertools import combinations

In [6]:
models = (['model007', 'model009', 'model011', 'model013', 'model015', 'model024', 'model026bm'])

In [7]:
# All combos of 7
combos = [c for c in combinations(models, 7)]
combos

[('model007',
  'model009',
  'model011',
  'model013',
  'model015',
  'model024',
  'model026bm')]

In [8]:
len(combos)

1

In [9]:
start_version = 139

for i in range(1):

    if start_version < 10:
        ensemble_version = 'ensemble00{}'.format(start_version)
    else:
        ensemble_version = 'ensemble0{}'.format(start_version)
        
    print('\nCreating {}:\n{}'.format(ensemble_version, combos[i]))
    _ = create_ensemble(combos[i], ensemble_version, use_orig_duration=True, save=True)
    start_version += 1


Creating ensemble0139:
('model007', 'model009', 'model011', 'model013', 'model015', 'model024', 'model026bm')


Evaluating the ensemble:   0%|          | 0/488 [00:00<?, ?it/s]

Computing results:


Evaluating the ensemble: 100%|██████████| 488/488 [08:29<00:00,  1.04s/it]
/home/js7561/miniconda3/envs/py35milsed/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/js7561/miniconda3/envs/py35milsed/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/js7561/miniconda3/envs/py35milsed/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/js7561/miniconda3/envs/py35milsed/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision i